# 实验课1：基于模板的手写数字识别

In [1]:
import pandas as pd
import numpy as np
import os
import cv2
from matplotlib import pyplot as plt

### 标准模板库制作

+ 灰度化

In [2]:
def graying(img):
    img =cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret,thresh=cv2.threshold(img,100,255,cv2.THRESH_BINARY)
    img = thresh
    return thresh

+ 基于matplotlib和jupyter的特殊opencv图像显示方式

In [3]:
def imshow(img):
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.subplot(122)
    plt.xticks([]), plt.yticks([])
    plt.imshow(img2)

+ 裁剪图片
  1. 利用findContours找到边缘
  2. 利用找到的边缘进行筛选和切割

In [4]:
def splitImg(img):
    #img = graying(img)
    contours,hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    for i in range(len(contours)):
        x,y,w,h = cv2.boundingRect(contours[i])
        #print([x,y,w,h])
    img = img[y:y+h,x:x+w]
    return img

+ 依据5x5标准进行标准化，得到标准化后的矩阵值
  + 计算每个单元格w与h时，采用对小数点后的位数四舍五入，并特殊处理最后一个单元格

In [5]:
def standard(img):
    w,h = img.shape[1],img.shape[0]
    w_i,h_i=[],[]
    n_grids = 10
    # 计算每一个单元格w与h的大小
    '''if w%5 != 0:
        if w%5 > 5:
            for i in range(4):w_i.append(w//5+1)
            w_i.append(w_i[0]*5-w+w//5)
        else:
            for i in range(4):w_i.append(w//5)
            w_i.append(w-w_i[0]*4)
    else:
        for i in range(5):w_i.append(w//5)
    
    if h%5 != 0:
        if h%5 > 5:
            for i in range(4):h_i.append(h//5+1)
            h_i.append(h_i[0]*5-h+h//5)
        else:
            for i in range(4):h_i.append(h//5)
            h_i.append(h-h_i[0]*4)
    else:
        for i in range(5):h_i.append(h//5)'''
    for i in range(n_grids):w_i.append(w//n_grids)
    for i in range(n_grids):h_i.append(w//n_grids)
    # 归一化
    loss = 0.2
    w_end,h_end = 0,0
    arr_matrix = []
    for i in range(n_grids):
        col = w_end
        w_end += w_i[i]
        h_end = 0
        for j in range(n_grids):
            count = 0
            count_a = 0
            row = h_end
            col = w_end-w_i[i]
            h_end += h_i[j]
            while col<w_end:
                row = h_end-h_i[j]
                while row<h_end:
                    if img[row,col] > 150:
                        
                        count +=1
                    count_a +=1
                    row +=1
                col +=1
            if count/count_a > loss:
                arr_matrix.append(1)
            else:
                arr_matrix.append(0)
            
    matrix = np.array(arr_matrix).reshape(n_grids,n_grids).T
    return matrix

+ 构造所有训练图像的标准库

In [60]:
#图片地址
def getTrain():
    local = './data/train-images/'
    if_train = 1
    if os.path.exists(r"./data/output.csv"):
        input_f_csv = pd.read_csv("./data/output.csv",index_col=0)
        #print(input_f_csv.shape)
        #print(input_f_csv)
        if input_f_csv.shape[0] == 10 and input_f_csv.shape[1] == 100:
            if_train = 0
            std_library = input_f_csv
        else:
            os.remove(r"./data/output.csv")
    if if_train:
        std_library = pd.DataFrame(columns=range(100))#1-->100
        for n in range(10):#1-->10
            arr_ =[]
            for m in range(100):#1-->100
                img = cv2.imread("%s%s_%s.bmp"%(local,n,m))
                img = graying(img)
                img = splitImg(img)
                img = cv2.resize(img,(50,50))
                ret,img = cv2.threshold(img,100,255,cv2.THRESH_BINARY)
                std_matrix = standard(img)
                str_std = ''
                for i in std_matrix.tolist():str_std = "%s,%s"%(str_std,",".join(map(str,i)))
                arr_.append(str_std[1:])
                # print(std_matrix)
            std_library.loc[n] = arr_

        #print(std_library)
        std_library.to_csv("./data/output.csv")
        #imshow(img)
        print("Start from creating.")
    else:
        print("Start from CSV file.")
    
    return std_library


# 转字符串为numpy矩阵
# np.array(list(map(int,str_std[1:].split(',')))).reshape(10,10)


### 读入测试集并与标准库比对

In [62]:
%%time
local_test = './data/test-images/'
dic_output = {}
std_library = getTrain()
for n in range(10):#1-->10
    for m in range(20):#1-->20
        img = cv2.imread("%s%s_%s.bmp"%(local_test,n,m))
        img = graying(img)
        img = splitImg(img)
        img = cv2.resize(img,(50,50))
        ret,img = cv2.threshold(img,100,255,cv2.THRESH_BINARY)
        train_matrix = standard(img)
        #print(train_matrix)
        for row in std_library.itertuples():
            hit = 0
            for i in range(1,101):#2-->101
                now_str = getattr(row,"_%s"%i)
                now_matrix = np.array(list(map(int,now_str.split(',')))).reshape(10,10)
                sub = now_matrix-train_matrix
                if np.sum(sub==0)/100 > hit: 
                    hit = np.sum(sub==0)/100
                if hit == 1.0:
                    break
            if hit == 1.0:
                str_name = "%s_%s"%(n,m)
                dic_output[str_name] = row.Index
                #print(row.Index)
                break
print(dic_output)

Start from CSV file.
{'0_0': 0, '0_1': 0, '0_2': 0, '0_3': 0, '0_4': 0, '0_5': 0, '0_6': 0, '0_7': 0, '0_8': 0, '0_9': 0, '0_10': 0, '0_11': 0, '0_12': 0, '0_13': 0, '0_14': 0, '0_15': 0, '0_16': 0, '0_17': 0, '0_18': 0, '0_19': 0, '1_0': 1, '1_1': 1, '1_2': 1, '1_3': 1, '1_4': 1, '1_5': 1, '1_6': 1, '1_7': 1, '1_8': 1, '1_9': 1, '1_10': 1, '1_11': 1, '1_12': 1, '1_13': 1, '1_14': 1, '1_15': 1, '1_16': 1, '1_17': 1, '1_18': 1, '1_19': 1, '2_0': 2, '2_1': 2, '2_2': 2, '2_3': 2, '2_4': 2, '2_5': 2, '2_6': 2, '2_7': 2, '2_8': 2, '2_9': 2, '2_10': 2, '2_11': 2, '2_12': 2, '2_13': 2, '2_14': 2, '2_15': 2, '2_16': 2, '2_17': 2, '2_18': 2, '2_19': 2, '3_0': 3, '3_1': 3, '3_2': 3, '3_3': 3, '3_4': 3, '3_5': 3, '3_6': 3, '3_7': 3, '3_8': 3, '3_9': 3, '3_10': 3, '3_11': 3, '3_12': 3, '3_13': 3, '3_14': 3, '3_15': 3, '3_16': 3, '3_17': 3, '3_18': 3, '3_19': 3, '4_0': 4, '4_1': 4, '4_2': 4, '4_3': 4, '4_4': 4, '4_5': 4, '4_6': 4, '4_7': 4, '4_8': 4, '4_9': 4, '4_10': 4, '4_11': 4, '4_12': 4, '4_13